In [50]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from geophires_x_client import GeophiresXClient
from geophires_x_client.geophires_input_parameters import GeophiresInputParameters
import numpy as np
from mpmath import *
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit

example_file_path = Path('examples/example3.txt').absolute()

client = GeophiresXClient()
result = client.get_geophires_result(
            GeophiresInputParameters(from_file_path=example_file_path)
        )

with open(result.output_file_path,'r') as f:
    print(f.read())

[2023-11-27 21:28:45][WARNING] Parameter given (400.0) for Maximum Temperature is being set by the input file to a value that is the same as the default. No change was made to that value. Recommendation: remove the Maximum Temperature from the input file unless you wish to change it from the default value of (400.0)
[2023-11-27 21:28:45][WARNING] Parameter given (1) for Number of Segments is being set by the input file to a value that is the same as the default. No change was made to that value. Recommendation: remove the Number of Segments from the input file unless you wish to change it from the default value of (1)
[2023-11-27 21:28:45][WARNING] Parameter given (1000.0) for Reservoir Heat Capacity is being set by the input file to a value that is the same as the default. No change was made to that value. Recommendation: remove the Reservoir Heat Capacity from the input file unless you wish to change it from the default value of (1000.0)
[2023-11-27 21:28:45][WARNING] Parameter given

In [52]:
import pprint

# Assuming 'result' is your GeophiresXResult object
all_results = result.result

# Dynamically creating variables for each category
for category in all_results.keys():
    globals()[category.replace(' ', '_').upper()] = all_results.get(category, {})

# Pretty-print the all_results
pprint.pprint(all_results)


{'CAPITAL COSTS (M$)': {'Auxiliary Heater Cost': None,
                        'District Heating System Cost': None,
                        'Drilling Cost': None,
                        'Drilling and Completion Costs': None,
                        'Drilling and Completion Costs per Well': None,
                        'Drilling and completion costs': {'unit': 'MUSD',
                                                          'value': 34.45},
                        'Drilling and completion costs per well': {'unit': 'MUSD',
                                                                   'value': 5.74},
                        'Exploration costs': {'unit': 'MUSD', 'value': 5.51},
                        'Field gathering system costs': {'unit': 'MUSD',
                                                         'value': 3.16},
                        'Pump Cost': None,
                        'Stimulation costs': {'unit': 'MUSD', 'value': 4.53},
                        'Surface power pl

In [ ]:
depth_m = all_results.get("ENGINEERING PARAMETERS", {}).get("Well depth (or total length, if not vertical)", {}).get('value', None)
number_of_prod_wells = all_results.get("ENGINEERING PARAMETERS", {}).get("Number of Production Wells", {}).get('value', None)
number_of_inj_wells = all_results.get("ENGINEERING PARAMETERS", {}).get("Number of Injection Wells", {}).get('value', None)
max_reservoir_temp = all_results.get("RESOURCE CHARACTERISTICS", {}).get("Maximum reservoir temperature", {}).get('value', None)

# CAPITAL COSTS - you need to adjust the keys based on the actual data
wellfield_cost = all_results.get("CAPITAL COSTS (M$)", {}).get("Drilling and completion costs", {}).get('value', None)
surface_plant_cost = all_results.get("CAPITAL COSTS (M$)", {}).get("Surface power plant costs", {}).get('value', None)
exploration_cost = all_results.get("CAPITAL COSTS (M$)", {}).get("Exploration costs", {}).get('value', None)
gathering_cost = all_results.get("CAPITAL COSTS (M$)", {}).get("Field gathering system costs", {}).get('value', None)

# OPERATING AND MAINTENANCE COSTS
wellfield_OM_cost = all_results.get("OPERATING AND MAINTENANCE COSTS (M$/yr)", {}).get("Wellfield maintenance costs", {}).get('value', None)
surface_plant_OM_cost = all_results.get("OPERATING AND MAINTENANCE COSTS (M$/yr)", {}).get("Power plant maintenance costs", {}).get('value', None)
water_OM_cost = all_results.get("OPERATING AND MAINTENANCE COSTS (M$/yr)", {}).get("Water costs", {}).get('value', None)

# SURFACE EQUIPMENT SIMULATION RESULTS
avg_total_heat_gen = all_results.get("SURFACE EQUIPMENT SIMULATION RESULTS", {}).get("Average Total Electricity Generation", {}).get('value', None)
avg_total_electricity_gen = all_results.get("SURFACE EQUIPMENT SIMULATION RESULTS", {}).get("Average Total Electricity Generation", {}).get('value', None)

# ENGINEERING PARAMETERS RESULTS
# efficiency = all_results.get("ENGINEERING PARAMETERS", {}).get("Pump efficiency", {}).get('value', None)

# ECONOMIC PARAMETERS RESULTS 
# interest = all_results.get("ECONOMIC PARAMETERS", {}).get("Interest Rate", {}).get('value', None)
lifetime = all_results.get("ECONOMIC PARAMETERS", {}).get("Project lifetime", {}).get('value', None)


In [ ]:
# Construct the DataFrame
df_final = pd.DataFrame({
    'Depth (m)': [depth_m],
    'Number of Prod Wells': [number_of_prod_wells],
    'Number of Inj Wells': [number_of_inj_wells],
    'Maximum Reservoir Temperature (deg.C)': [max_reservoir_temp],
    'Wellfield Cost ($M)': [wellfield_cost],
    'Surface Plant Cost ($M)': [surface_plant_cost],
    'Exploration Cost ($M)': [exploration_cost],
    'Field Gathering System Cost ($M)': [gathering_cost],
    'Wellfield O&M Cost ($M/year)': [wellfield_OM_cost],
    'Surface Plant O&M Cost ($M/year)': [surface_plant_OM_cost],
    'Make-Up Water O&M Cost ($M/year)': [water_OM_cost],
    'Average Reservoir Heat Extraction (MWth)': [avg_total_heat_gen],
    'Average Total Electricity Generation (MWe)': [avg_total_electricity_gen],
    # 'Efficiency':[efficiency],
    # 'Interest Rate':[interest],
    'Lifetime':[lifetime]
})


In [ ]:
# Sort the DataFrame by specified columns
df_final = df_final.sort_values(by=['Depth (m)', 'Number of Prod Wells', 'Number of Inj Wells'], ascending=[True, True, True])

# Export to Excel
# Replace {plant} with the actual variable or string representing the plant name
df_final.to_csv("results.csv")

In [ ]:
plant = "CHP"
### Scatterplot 
### LINE FIT
df_line = df_final
df_line = pd.concat([df_line, pd.Series(0, index=df_line.columns)], ignore_index=True)

max_e_cap               = np.max(df_line['Average Total Electricity Generation (MWe)'])     ####ADDEDD
max_h_cap               = np.max(df_line['Average Reservoir Heat Extraction (MWth)'])       ####ADDEDD
# plant_efficiency        = np.average(df_line['Efficiency'])                                 ####ADDEDD
# lifecycle               = df_line['Lifetime'][1]                                            ####ADDEDD
# rate                    = df_line['Interest Rate'][1]                                       ####ADDEDD

thermal_capacity_line       = np.array(df_line['Average Reservoir Heat Extraction (MWth)'])
electric_capacity_line      = np.array(df_line['Average Total Electricity Generation (MWe)'])
subsurface_cost_line        = np.add(np.array(df_line['Wellfield Cost ($M)']),
                        # np.array(df_final['Exploration Cost ($M)']),
                        np.array(df_line['Field Gathering System Cost ($M)']))
surface_cost_line           = np.array(df_line['Surface Plant Cost ($M)'])
subsurface_o_m_cost_line    = np.add(np.array(df_line['Wellfield O&M Cost ($M/year)']),
                        np.array(df_line['Make-Up Water O&M Cost ($M/year)']))
surface_o_m_cost_line       = np.array(df_line['Surface Plant O&M Cost ($M/year)'])

###Scatter
thermal_capacity            = np.array(df_final['Average Reservoir Heat Extraction (MWth)'])
electric_capacity           =  np.array(df_final['Average Total Electricity Generation (MWe)'])
subsurface_cost             = np.add(np.array(df_final['Wellfield Cost ($M)']),
                        # np.array(df_final['Exploration Cost ($M)']),
                        np.array(df_final['Field Gathering System Cost ($M)']))
surface_cost                = np.array(df_final['Surface Plant Cost ($M)'])
subsurface_o_m_cost         = np.add(np.array(df_final['Wellfield O&M Cost ($M/year)']),
                        np.array(df_final['Make-Up Water O&M Cost ($M/year)']))
surface_o_m_cost            = np.array(df_final['Surface Plant O&M Cost ($M/year)'])

In [ ]:

def fit_linear_model(x, y):
    def objective(x, a, b):
        return a * x + b

    popt, _ = curve_fit(objective, x, y)
    a, b = popt
    x_line = np.asarray([np.min(x), np.max(x)])
    b_values = y - np.multiply(a, x)
    lower_b = np.percentile(b_values, 5)  # 10th percentile as lower bound
    lower_line = objective(x_line, a, lower_b)
    label = f"y={a:.4f}x+{lower_b:.4f}"
    
    return a, lower_b, x_line, lower_line, label

# Example usage:
x1              = thermal_capacity
y1              = subsurface_cost
a1, b1, x1_line, lower_b1_line, label_b1 = fit_linear_model(thermal_capacity, subsurface_cost)

x2              = electric_capacity
y2              = surface_cost
a2, b2, x2_line, lower_b2_line, label_b2 = fit_linear_model(electric_capacity, surface_cost)

x3              = thermal_capacity
y3              = subsurface_o_m_cost
a3, b3, x3_line, lower_b3_line, label_b3 = fit_linear_model(thermal_capacity, subsurface_o_m_cost)

x4              = electric_capacity
y4              = surface_o_m_cost  
a4, b4, x4_line, lower_b4_line, label_b4 = fit_linear_model(electric_capacity, surface_o_m_cost)

In [ ]:

#find line of best fit

plt.rcParams["figure.figsize"] = [7.00, 3.0]
plt.rcParams["figure.autolayout"] = True    
# define color map
cmap = plt.get_cmap('OrRd')

# find unique values of Number of Prod Wells
unique_prod_wells = df_final['Number of Prod Wells'].unique()

# Plot1
# plt.scatter(x1,y1,color='green',label="Raw data",s=4)
# create scatter plot for each unique value of Number of Prod Wells
fig1 = plt.figure()
for i, prod_wells in enumerate(unique_prod_wells):
    mask = df_final['Number of Prod Wells'] == prod_wells
    x = x1[mask]
    y = y1[mask]
    plt.scatter(x, y, c=cmap(i/len(unique_prod_wells)), label=prod_wells, s=4)
plt.plot(x1_line, lower_b1_line, '--', color='red', label=label_b1)
plt.title(f'{plant} subsurface cost-to-thermal capacity relation')
plt.xlabel('Avg. Thermal capacity (MWth)')
plt.ylabel('Subsurface Total Cost ($M)')
plt.legend(handles=[plt.plot([], [], c='red', ls='--', label=label_b1)[0]])
#add line of best fit to plot
#add fitted regression equation to plot

fig2 = plt.figure()
# Plot2
# plt.scatter(x2,y2,color='blue',label="Raw data",s=4)
for i, prod_wells in enumerate(unique_prod_wells):
    mask = df_final['Number of Prod Wells'] == prod_wells
    x = x2[mask]
    y = y2[mask]
    plt.scatter(x, y, c=cmap(i/len(unique_prod_wells)), label=prod_wells, s=4)
plt.plot(x2_line, lower_b2_line, '--', color='red', label=label_b2)
plt.title(f'{plant} surface cost-to-electric capacity relation')
plt.xlabel('Avg. Electric capacity (MWe)')
plt.ylabel('Surface Total Cost ($M)')
# plt.legend()
plt.legend(handles=[plt.plot([], [], c='red', ls='--', label=label_b2)[0]])

fig3 = plt.figure()
# plt.scatter(x3,y3,color='pink',label="Raw data",s=4)
for i, prod_wells in enumerate(unique_prod_wells):
    mask = df_final['Number of Prod Wells'] == prod_wells
    x = x3[mask]
    y = y3[mask]
    plt.scatter(x, y, c=cmap(i/len(unique_prod_wells)), label=prod_wells, s=4)
plt.plot(x3_line, lower_b3_line, '--', color='red', label=label_b3)
plt.title(f'{plant} subsurface O&M cost-to-thermal capacity relation')
plt.xlabel('Avg. Thermal capacity (MWth)')
plt.ylabel('Subsurface Total O&M Cost ($M)')
# plt.legend()
plt.legend(handles=[plt.plot([], [], c='red', ls='--', label=label_b3)[0]])

#add line of best fit to plot
#add fitted regression equation to plot

fig4 = plt.figure()
# plt.scatter(x4,y4,color='orange',label="Raw data",s=4)
for i, prod_wells in enumerate(unique_prod_wells):
    mask = df_final['Number of Prod Wells'] == prod_wells
    x = x4[mask]
    y = y4[mask]
    plt.scatter(x, y, c=cmap(i/len(unique_prod_wells)), label=prod_wells, s=4)
plt.plot(x4_line, lower_b4_line, '--', color='red', label=label_b4)
plt.title(f'{plant} surface O&M cost-to-electric capacity relation')
plt.xlabel('Avg. Electric capacity (MWe)')
plt.ylabel('Surface O&M Total Cost ($M)')
# plt.legend()
plt.legend(handles=[plt.plot([], [], c='red', ls='--', label=label_b4)[0]])


filename = f"/Users/bpulluta/GEOPHIRES-v2/DEMO/results/{plant}/final_{plant}_results.pdf" 

def save_image(filename):
    p = PdfPages(filename)
    fig_nums = plt.get_fignums()  
    figs = [plt.figure(n) for n in fig_nums]
    for fig in figs: 
        fig.savefig(p, format='pdf') 
    p.close()  
    
save_image(filename)  


In [ ]:
# Assuming 'result' is your GeophiresXResult object
summary_of_results = result.result.get('SUMMARY OF RESULTS', {})

# Convert to JSON and save to file
json_data = json.dumps(summary_of_results, indent=4)
with open('summary_of_results.json', 'w') as file:
    file.write(json_data)
    
# Load JSON data
with open('summary_of_results.json', 'r') as file:
    data = json.load(file)

# Process data to extract numeric values
processed_data = {}
for key, value in data.items():
    if isinstance(value, dict) and 'value' in value:
        processed_data[key] = value['value']
    else:
        processed_data[key] = None  # or some default value, e.g., 0

# Convert processed data to DataFrame
df = pd.DataFrame([processed_data])

# Drop columns with None values for cleaner plotting
df = df.dropna(axis=1)

plt.figure(figsize=(12, 6))
bars = plt.bar(df.columns, df.iloc[0])
plt.xlabel('Metrics')
plt.ylabel('Values')
plt.title('Summary of Results - GeophiresX')
plt.xticks(rotation=45)

# Annotate each bar with its value
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), 
             va='bottom', ha='center')

plt.show()

In [ ]:
well_depth = all_results.get("ENGINEERING PARAMETERS", {}).get("Well depth (or total length, if not vertical)", None)
print(well_depth['value'])